### Open Flights Data Wrangling

To practice, you are going to wrangle data from OpenFlights.  You can read about it here: 

http://openflights.org/data.html

This includes three main files, one for each airport, one for each airline, and one for each route.  They can be merged or joined with the appropriate fields.  I have modified the files slightly to include a header row in the .dat files, which makes it a bit easier for you.  

You are required to work through the problems below.  This may take some time.  Be persistent, and ask questions or seek help as needed.  

In [2]:
import pandas as pd
import numpy as np

In [3]:
# These files use \N as a missing value indicator.  When reading the CSVs, we will tell
# it to use that value as missing or NA.  The double backslash is required because
# otherwise it will interpret \N as a carriage return. 

# Read in the airports data.
airports = pd.read_csv("data/airports.dat", header=None, na_values='\\N')
airports.columns = ["id", "name", "city", "country", "iata", "icao", "latitude", "longitude", "altitude","timezone", "dst", "tz", "type", "source"]

# Read in the airlines data.
airlines = pd.read_csv("data/airlines.dat", header=None, na_values='\\N')
airlines.columns = ["id", "name", "alias", "iata", "icao", "callsign", "country", "active"]

# Read in the routes data.
routes = pd.read_csv("data/routes.dat", header=None, na_values='\\N')
routes.columns = ["airline", "airline_id", "source", "source_id", "dest", "dest_id", "codeshare", "stops", "equipment"]

1) Start by seeing what's in the data.  What columns are there?  What data types are the columns?  

Remember, 'object' means it is a string, while the numerical values can be floats or ints.  Sometimes you will have problems if it reads numeric data in as strings.  If that happens, you can use the function .astype() to convert it.  Look it up in the pandas API to get more details

In [17]:
print(airports.dtypes)
airports.head()

id             int64
name          object
city          object
country       object
iata          object
icao          object
latitude     float64
longitude    float64
altitude       int64
timezone     float64
dst           object
tz            object
type          object
source        object
dtype: object


,id,name,city,country,iata,icao,latitude,longitude,altitude,timezone,dst,tz,type,source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10.0,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10.0,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10.0,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10.0,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10.0,U,Pacific/Port_Moresby,airport,OurAirports


In [18]:
print(airlines.dtypes)
airlines.head()

id           int64
name        object
alias       object
iata        object
icao        object
callsign    object
country     object
active      object
dtype: object


,id,name,alias,iata,icao,callsign,country,active
0,-1,Unknown,NaN,-,NaN,NaN,NaN,Y
1,1,Private flight,NaN,-,NaN,NaN,NaN,Y
2,2,135 Airways,NaN,NaN,GNL,GENERAL,United States,N
3,3,1Time Airline,NaN,1T,RNX,NEXTIME,South Africa,Y
4,4,2 Sqn No 1 Elementary Flying Training School,NaN,NaN,WYT,NaN,United Kingdom,N


In [21]:
print(routes.dtypes)
routes.head()

airline        object
airline_id    float64
source         object
source_id     float64
dest           object
dest_id       float64
codeshare      object
stops           int64
equipment      object
dtype: object


,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,2B,410.0,AER,2965.0,KZN,2990.0,NaN,0,CR2
1,2B,410.0,ASF,2966.0,KZN,2990.0,NaN,0,CR2
2,2B,410.0,ASF,2966.0,MRV,2962.0,NaN,0,CR2
3,2B,410.0,CEK,2968.0,KZN,2990.0,NaN,0,CR2
4,2B,410.0,CEK,2968.0,OVB,4078.0,NaN,0,CR2


2) Select just the routes that go to or from Lexington Bluegrass Airport, and store them in their own dataframe.  

The airport code is LEX.  You should have a much smaller dataframe.  How many inbound routes and how many outbound routes are there? 

In [31]:
routes_lex = routes[((routes["source"]=="LEX") | (routes["dest"]=="LEX"))]
routes_lex.head()

,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
3569,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ
3588,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ
4953,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7
5247,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4
5763,AA,24.0,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ


In [36]:
print((routes_lex["source"]=="LEX").value_counts())
print((routes_lex["dest"]=="LEX").value_counts())

True     20
False    20
Name: source, dtype: int64
True     20
False    20
Name: dest, dtype: int64


3) Now let's look at which airlines operate in and out of Lexington.  To do this, you need to merge the airline dataframe to the route dataframe.  

How many routes does each airline have?  The value_counts() method may be useful for answering this question.  

In [54]:
#merge dataframes
airline_merge_route = pd.merge(routes,airlines, left_on = "airline_id", right_on = "id")

#filter for Lexington
airline_merge_route_lex = airline_merge_route[((airline_merge_route["source"] == "LEX") | (airline_merge_route["dest"]=="LEX"))]
airline_merge_route_lex.head()

,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,id,name,alias,iata,icao,callsign,country,active
3443,9E,3976,ATL,3682.0,LEX,4017.0,NaN,0,CRJ,3976,Pinnacle Airlines,NaN,9E,FLG,FLAGSHIP,United States,Y
3462,9E,3976,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,3976,Pinnacle Airlines,NaN,9E,FLG,FLAGSHIP,United States,Y
4781,AA,24,CLT,3876.0,LEX,4017.0,NaN,0,CR7,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y
5075,AA,24,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y
5591,AA,24,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y


In [61]:
count = airline_merge_route_lex.groupby("name")["source"].nunique()
count

name
Air France                  2
Allegiant Air               5
American Airlines           4
Delta Air Lines             6
KLM Royal Dutch Airlines    2
Pinnacle Airlines           2
US Airways                  4
United Airlines             3
Name: source, dtype: int64

In [62]:
airline_merge_route_lex["country"].unique()

array(['United States', 'France', 'Netherlands'], dtype=object)

4) It looks like there are some international airlines with Lexington routes.  To look at how many routes they have, create a new column in your dataframe called 'International', which is set to Y for an overseas airline and N for a domestic airline.  Calculate the percent of routes with an overseas airline.  

In [67]:
airline_merge_route_lex['International'] = np.where(airline_merge_route_lex["country"]!="United States", "Y", "N")
airline_merge_route_lex

C:\Users\Michael\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,id,name,alias,iata,icao,callsign,country,active,International
3443,9E,3976,ATL,3682.0,LEX,4017.0,NaN,0,CRJ,3976,Pinnacle Airlines,NaN,9E,FLG,FLAGSHIP,United States,Y,N
3462,9E,3976,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,3976,Pinnacle Airlines,NaN,9E,FLG,FLAGSHIP,United States,Y,N
4781,AA,24,CLT,3876.0,LEX,4017.0,NaN,0,CR7,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,N
5075,AA,24,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,N
5591,AA,24,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,N
5592,AA,24,LEX,4017.0,DFW,3670.0,Y,0,ERD ER4,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,N
5593,AA,24,LEX,4017.0,ORD,3830.0,Y,0,ERD ER4,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,N
6111,AA,24,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,N
8925,AF,137,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717,137,Air France,NaN,AF,AFR,AIRFRANS,France,Y,Y
9469,AF,137,LEX,4017.0,ATL,3682.0,Y,0,CRJ CR9,137,Air France,NaN,AF,AFR,AIRFRANS,France,Y,Y


In [78]:
100*(airline_merge_route_lex['International'].value_counts())/(len(airline_merge_route_lex))

N    90.0
Y    10.0
Name: International, dtype: float64

5) Actually, it looks like a bunch of these routes are codeshares.  That means they are marketed by this airline, but operated by a different airline.  See the note in the data documentation on openflights.org/data.  The implication of this is that there are duplicates.

Can you figure out which ones are duplicates?  Can you then create a dataframe with only the unique routes?  How many unique inbound and outbound routes are there? 

Remember, someone has to operate the flight, so if all the routes to/from a particular airport are listed as codeshares, then something is funny...

It is also possible that more than one airline actually operates a route between the same two airports. (Having this sort of competition generally means that you will get better fares as a traveler.)  It may not be obvious what is actually in the data set, so dig or do external research as needed.  

In [82]:
airline_merge_route_lex_nocodeshare = airline_merge_route_lex[airline_merge_route_lex["codeshare"]!="Y"]
airline_merge_route_lex_nocodeshare

,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,id,name,alias,iata,icao,callsign,country,active,International
3443,9E,3976,ATL,3682.0,LEX,4017.0,NaN,0,CRJ,3976,Pinnacle Airlines,NaN,9E,FLG,FLAGSHIP,United States,Y,N
3462,9E,3976,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,3976,Pinnacle Airlines,NaN,9E,FLG,FLAGSHIP,United States,Y,N
4781,AA,24,CLT,3876.0,LEX,4017.0,NaN,0,CR7,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,N
19992,DL,2009,ATL,3682.0,LEX,4017.0,NaN,0,M88 717,2009,Delta Air Lines,NaN,DL,DAL,DELTA,United States,Y,N
20466,DL,2009,DTW,3645.0,LEX,4017.0,NaN,0,717,2009,Delta Air Lines,NaN,DL,DAL,DELTA,United States,Y,N
20923,DL,2009,LEX,4017.0,ATL,3682.0,NaN,0,M88 717,2009,Delta Air Lines,NaN,DL,DAL,DELTA,United States,Y,N
20926,DL,2009,LEX,4017.0,LGA,3697.0,NaN,0,ERJ,2009,Delta Air Lines,NaN,DL,DAL,DELTA,United States,Y,N
28774,G4,35,FLL,3533.0,LEX,4017.0,NaN,0,M80,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,N
28870,G4,35,LEX,4017.0,FLL,3533.0,NaN,0,M80,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,N
28871,G4,35,LEX,4017.0,PGD,7056.0,NaN,0,M80,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,N
